# Многопоточность в Python. Библиотеки threading и multiprocessing.

http://cs.mipt.ru/advanced_python/lessons/lab6.html

## Задание 1:

распараллелить чистку текста (колонки text). Функция чистки текста уже написана за вас. Вам нужно только установить все библиотеки (например, через анаконду)


https://anaconda.org/anaconda/nltk

In [2]:
# подрубаем нужные библиотеки
import os # для путей
import numpy as np # для массивов
import pandas as pd # для работы и хранения данных в ОП

import warnings 
warnings.filterwarnings('ignore') # все предупреждения офф - чтобы не мешали

from nltk.corpus import stopwords # частые слова
from nltk.stem import PorterStemmer # нлтк стеммер
from nltk.stem import WordNetLemmatizer # нлтк лемматизатор
from nltk import word_tokenize, sent_tokenize # для разбиения предложений на токены-слова
import re # библиотека для регулярных выражений

данные здесь: https://www.kaggle.com/c/dlinnlp-spring-2019-clf/data

In [3]:
train_df = pd.read_csv ('train.csv')
test_df = pd.read_csv ('test.csv')
df = pd.concat([train_df, test_df], axis=0, ignore_index=True).drop(['Unnamed: 0', 'label'], axis=1)
print(len(train_df), len(test_df), len(df))
df.head(10)

24871 5647 30518


,text,title
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat..."
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya..."


In [4]:
# функция для очистки текста

def clean_text(data):
    low = True 
    delstops = True
    lemm = True
    stem = True
    del12gram = True  # True = убираем обрывки слов в 1-2 символа
    
    
    stops = set(stopwords.words("english"))
    data = str(data)
    data = re.sub(r'[^a-zA-Z ]',r' ',data)
    data = ' '.join(data.split()) #убираем лишние пробелы
    
    if low:
        data = data.lower()
    
    if delstops:
        data = " ".join([w for w in data.split() if w not in stops])

    if lemm:      
        wordnet_lemmatizer = WordNetLemmatizer()
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='a') for w in data.split()])
        data = " ".join([wordnet_lemmatizer.lemmatize(w, pos='n') for w in data.split()])
        
    if stem:
        st = PorterStemmer()
        data = " ".join([st.stem(w) for w in data.split()])
        
    if del12gram:
        data = " ".join([w for w in data.split() if len(w) > 2])  
          
    return data

In [10]:
%%time
# применяем функцию очистки текста
df['cleaned_text'] = df['text'].map(clean_text) # ~ Wall time: 5min 34s

Wall time: 10min 19s


In [18]:
df.head(10)

,text,title,cleaned_text
0,Economists generally agree: China must overhau...,China and Economic Reform: Xi Jinping’s Track ...,economist gener agre china must overhaul huge ...
1,LONDON—British Prime Minister Theresa May said...,Trade to Be a Big Topic in Theresa May’s U.S. ...,london british prime minist theresa may said d...
2,Beaches come in all sorts of shapes and sizes ...,"The Top Beaches In The World, According To Nat...",beach come sort shape size beyond typic caribb...
3,"A timeline of what happened after Tamir Rice, ...",Sheriff’s Report Provides New Details on Tamir...,timelin happen tamir rice year old boy kill po...
4,An Italian neurosurgeon who has claimed for mo...,Surgeon claiming he will transplant volunteer'...,italian neurosurgeon claim month perform world...
5,Spanx does not do the thing I thought it did. ...,This Is How Differently Priced Spanx Can Actua...,spanx thing thought kristin like mani ladi lil...
6,"“Jane Pauley! You’re all in leather,” said Sam...",Samantha Bee and Jane Pauley Are Breaking the ...,jane pauley leather said samantha bee pauley w...
7,Charles Krauthammer called President Trump's 5...,Krauthammer: Syria Strike 'Total Contradiction...,charl krauthamm call presid trump missil strik...
8,The United States is making more things than e...,Rust Belt voters made Trump president. Now the...,unit state make thing ever read right manufact...
9,As Jewish settlers and protesters were removed...,"As Illegal Outpost Cleared, Israeli PM Netanya...",jewish settler protest remov illeg outpost occ...


In [40]:
import threading

In [23]:
length = df['text'].shape[0]

In [42]:
ind = [range(0, length//4), range(length//4, length//2), range(length//2, 3*length//4), range(3*length//4, length)]

In [37]:
def thread_job(number, data):
    df.iloc[ind[number], 3] = list(map(clean_text, data))
    print(f'Thread {number}/4 is done')

In [38]:
def run_threads():
    threads = [threading.Thread(target=thread_job, args=(i,df['text'].values[ind[i]],)) for i in range(4)]
    for thread in threads:
        thread.start() 
    for thread in threads:
        thread.join()

In [43]:
%%time
run_threads()

Exception in thread Thread-11:
Traceback (most recent call last):
  File "C:\Users\Lisa\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Lisa\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-37-bb0cc18b3311>", line 2, in thread_job
    df.iloc[ind[number], 3] = list(map(clean_text, data))
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 188, in __setitem__
    indexer = self._get_setitem_indexer(key)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 166, in _get_setitem_indexer
    return self._convert_tuple(key, is_setter=True)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 247, in _convert_tuple
    idx = self._convert_to_indexer(k, axis=i, is_setter=is_setter)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 2119, in _convert_to_indexer
    self._val

Wall time: 5min 4s


Exception in thread Thread-10:
Traceback (most recent call last):
  File "C:\Users\Lisa\Anaconda3\lib\threading.py", line 917, in _bootstrap_inner
    self.run()
  File "C:\Users\Lisa\Anaconda3\lib\threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-37-bb0cc18b3311>", line 2, in thread_job
    df.iloc[ind[number], 3] = list(map(clean_text, data))
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 188, in __setitem__
    indexer = self._get_setitem_indexer(key)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 166, in _get_setitem_indexer
    return self._convert_tuple(key, is_setter=True)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 247, in _convert_tuple
    idx = self._convert_to_indexer(k, axis=i, is_setter=is_setter)
  File "C:\Users\Lisa\Anaconda3\lib\site-packages\pandas\core\indexing.py", line 2119, in _convert_to_indexer
    self._val

# Задание 2

Иногда бывает нужно узнать доступность набора ip адресов. Неэффективный вариант представлен ниже.

Реализуйте то же самое, но используя threading.

In [49]:
%%time

received_packages = re.compile(r"(\d) received")
status = ("no response", "alive but losses", "alive")

for suffix in range(20, 30):
    ip = "192.168.178." + str(suffix)
    ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
    print("... pinging ", ip)
    while True:
        line = ping_out.readline()
        if not line:
            break
        n_received = received_packages.findall(line)
        if n_received:
            print(ip + ": " + status[int(n_received[0])])

... pinging  192.168.178.20
... pinging  192.168.178.21
... pinging  192.168.178.22
... pinging  192.168.178.23
... pinging  192.168.178.24
... pinging  192.168.178.25
... pinging  192.168.178.26
... pinging  192.168.178.27
... pinging  192.168.178.28
... pinging  192.168.178.29
Wall time: 523 ms


In [54]:
def thread_job(number, _range):
    received_packages = re.compile(r"(\d) received")
    status = ("no response", "alive but losses", "alive")

    for suffix in _range:
        ip = "192.168.178." + str(suffix)
        ping_out = os.popen("ping -q -c2 " + ip, "r")  # получение вердикта
        print("... pinging ", ip)
        while True:
            line = ping_out.readline()
            if not line:
                break
            n_received = received_packages.findall(line)
            if n_received:
                print(ip + ": " + status[int(n_received[0])])

ranges = [range(20,25),range(25,30)]
def run_threads(count):
    threads = [
        threading.Thread(target=thread_job, args=(i,ranges[i]))
        for i in range(0, count)
    ]
    for thread in threads:
        thread.start() 
    for thread in threads:
        thread.join()
        

In [53]:
%%time

run_threads(2)

... pinging  192.168.178.20
... pinging  192.168.178.25
... pinging  192.168.178.21
... pinging  192.168.178.26
... pinging  192.168.178.22
... pinging  192.168.178.27
... pinging  192.168.178.23
... pinging  192.168.178.28
... pinging ... pinging  192.168.178.24
 192.168.178.29
Wall time: 727 ms


# Задание 3

= Упражнение №4 из лабы про мультипроцессинг

In [56]:
import urllib.request
import time


urls = [
    'https://www.yandex.ru', 'https://www.google.com',
    'https://habrahabr.ru', 'https://www.python.org',
    'https://isocpp.org',
]


def read_url(url):
    with urllib.request.urlopen(url) as u:
        return u.read()


start = time.time()
for url in urls:
    read_url(url)
print(time.time() - start)

4.49766731262207


In [57]:
def thread_job(number, url):
    read_url(url)
def run_threads():
    threads = [
        threading.Thread(target=thread_job, args=(i,urls[i]))
        for i in range(0, 5)
    ]
    for thread in threads:
        thread.start() 
    for thread in threads:
        thread.join()

In [58]:
%%time
run_threads()

Wall time: 1.56 s


In [59]:
from multiprocessing import Process

In [60]:
def run_processes():    
    processes = [
        Process(target=read_url, args=(urls[i],))
        for i in range(0, 5)
    ]
    for proc in processes:
        proc.start() 
    for proc in processes:
        proc.join()

In [61]:
%%time
run_processes()

Wall time: 744 ms


дедлайн по ДЗ8 - вечер следующего вс 3.11.2019 23:59. Удачи!